# Predicting House Sale Prices
In this project we'll with housing data for the city of Ames, Iowa, United States from 2006 to 2010. The information about the data can be found [here](https://doi.org/10.1080/10691898.2011.11889627). You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

## Introduction
Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
df.head(5)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [2]:
def transform_features(df):
    return(df)

def select_features(df):
    return(df[['Gr Liv Area','SalePrice']])

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    #numeric columns except SalePrice
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    prediction = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], prediction)
    rmse = np.sqrt(mse)
    return(rmse)

transform_df = transform_features(df)
filtered_df = select_features(df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

## Feature Engineering
Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. 

* For now, let's drop any column with 5% or more missing values for now.

In [3]:
num_missing = df.isnull().sum()

In [4]:
drop_missing_cols = num_missing[(num_missing > len(df)*0.05)].sort_values()
drop_missing_cols

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

In [5]:
df = df.drop(drop_missing_cols.index, axis=1)
df.head(5)

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


* We'll drop any text column with 1 or more missing values.

In [6]:
text_mv_counts = df.select_dtypes('object').isnull().sum().sort_values(ascending=False)

drop_missing_cols2 = text_mv_counts[text_mv_counts > 0]

df = df.drop(drop_missing_cols2.index, axis=1)
df.head(5)

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


* Let's fill the missing values of  the Numerical columns with the most common value in the column.

In [7]:
numeric_mv = df.select_dtypes(["integer","float"]).isnull().sum().sort_values(ascending=False)
missing_cols3 = numeric_mv[numeric_mv>0] 
missing_cols3

Mas Vnr Area      23
Bsmt Half Bath     2
Bsmt Full Bath     2
Garage Cars        1
BsmtFin SF 1       1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 2       1
Garage Area        1
dtype: int64

In [8]:
# Computing the most common value for each column in `missing_cols3`
replacement_values_dict = df[missing_cols3.index].mode().to_dict(orient='records')[0]

In [9]:
## Using `pd.DataFrame.fillna()` to replace missing values.
df = df.fillna(replacement_values_dict)

In [10]:
df.isnull().sum().value_counts()

0    64
dtype: int64

Let's find what new features we can create for better capture of the information in some of the features? (years_until_remod can be an example)

According to the documentation, below columns look attractive for such manipulations.
* Year Built: Original construction date
* Year Remod/Add: Remodel date (same as construction date if no remodeling or additions)
* Yr Sold: Year Sold (YYYY)



In [11]:
years_sold = df['Yr Sold'] - df['Year Built']

'years_sold' column will represent the difference between Year Sold (the later value) and Year Built (the earlier value). Logically the value shall be positive so let's identify any column having negative value (we might drop them later). 

In [12]:
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [13]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']

'years_since_remod' column will represent the difference between Year Sold (the later value) and Year Remod/Add (the earlier value). Logically the value shall be positive so let's identify any column having negative value (we might drop them later).

In [14]:
years_since_remod[years_since_remod<0]

1702   -1
2180   -2
2181   -1
dtype: int64

Let's add new columns and drop the ones original ones.
Also, we can drop the lines with negative values.

In [15]:
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

df = df.drop([1702, 2180, 2181], axis=0)

df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

Below we'll drop the columns which are unnecesary for ML and leak data about the final sale.  

In [16]:
df = df.drop(['PID', 'Order'], axis=1)

df = df.drop(['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)

Let's update our transform_features function.

In [17]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)*0.05)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes('object').isnull().sum().sort_values(ascending=False)
    drop_missing_cols2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols2.index, axis=1)
    
    numeric_mv = df.select_dtypes(["integer","float"]).isnull().sum().sort_values(ascending=False)
    missing_cols3 = numeric_mv[numeric_mv>0]
    replacement_values_dict = df[missing_cols3.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(['Year Built', 'Year Remod/Add', 'PID', 'Order',
                 'Mo Sold', 'Sale Condition', 'Sale Type'], axis = 1)
    
    return(df)

def select_features(df):
    return(df[['Gr Liv Area','SalePrice']])

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    #numeric columns except SalePrice
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    prediction = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], prediction)
    rmse = np.sqrt(mse)
    return(rmse)

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

## Feature Selection
Now that the data is cleaned and transformed, it's time to move on to feature selection for numerical features.

In [18]:
numerical_df = transform_df.select_dtypes(['integer', 'float'])
numerical_df.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,62,0,0,0,0,0,2010,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,0,0,0,120,0,0,2010,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,36,0,0,0,0,12500,2010,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,2010,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,34,0,0,0,0,0,2010,189900,13,12


In [19]:
abs_corr_coefs = numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coefs

BsmtFin SF 2         0.006127
Misc Val             0.019273
Yr Sold              0.030358
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

Let's keep the columns with a correlation coefficient larger than 0.5.

In [20]:
abs_corr_coefs[abs_corr_coefs>0.5]

Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [21]:
transform_df = transform_df.drop(abs_corr_coefs[abs_corr_coefs<0.5].index, axis=1)

Which columns in the data frame should be converted to the categorical data type? 

Let's first create a list of the columns which are meant to be categorical. 

In [22]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

Some of the columns were deleted in the way, so let's find the ones remained. 

In [23]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

How many unique values are there in each categorical column?

In [24]:
uniqueness_counts = transform_df[transform_cat_cols].apply(
    lambda col:len(col.value_counts())).sort_values()
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

Let's drop the columns with more than 10 unique values. 

In [25]:
transform_df = transform_df.drop(
    uniqueness_counts[uniqueness_counts>10].index, axis=1)
transform_df.head(5)

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Condition 1,Condition 2,Bldg Type,...,Gr Liv Area,Full Bath,Kitchen Qual,Functional,Garage Cars,Garage Area,Paved Drive,SalePrice,Years Before Sale,Years Since Remod
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,1656,1,TA,Typ,2.0,528.0,P,215000,50,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,1Fam,...,896,1,TA,Typ,1.0,730.0,Y,105000,49,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,1329,1,Gd,Typ,1.0,312.0,Y,172000,52,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,2110,2,Ex,Typ,2.0,522.0,Y,244000,42,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,1629,2,TA,Typ,2.0,482.0,Y,189900,13,12


We'll convert remaining text (object) columns to categorical and add new columns to the dataset. 

In [26]:
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col]=transform_df[col].astype('category')
    
transform_df = pd.concat([transform_df,
                         pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)

Let's update the logic for the select_features() function. This function should take in the new, modified train and test data frames that were returned from transform_features().

Also, for the final part of the pipeline, training and testing. Let's add a parameter named k that controls the type of cross validation that occurs.

In [27]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)*0.05)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols2.index, axis=1)
    
    numeric_mv = df.select_dtypes(include=["integer","float"]).isnull().sum().sort_values(ascending=False)
    missing_cols3 = numeric_mv[numeric_mv>0]
    replacement_values_dict = df[missing_cols3.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(['Year Built', 'Year Remod/Add', 'PID', 'Order',
                 'Mo Sold', 'Sale Condition', 'Sale Type'], axis = 1)
    
    return(df)

def select_features(df, coeff_threshold=0.5, uniq_threshold=10):
    numerical_df = df.select_dtypes(['integer', 'float'])
    abs_corr_coefs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coefs[abs_corr_coefs<coeff_threshold].index, axis=1)
   
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    
    uniqueness_counts = df[transform_cat_cols].apply(lambda col:len(col.value_counts())).sort_values()
    df = df.drop(uniqueness_counts[uniqueness_counts>uniq_threshold].index, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col]=df[col].astype('category')
    
    df = pd.concat([df,pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return(df)

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k==0:
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)
        
        return(rmse)
    
    elif k == 1:
        # Randomizing *all* rows from dataset and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return(avg_rmse)
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse


[27245.805432627472, 28454.259993101306, 37483.848715393986, 29265.30594967173]


30612.305022698627